In [ ]:
import os
from os import mkdir
import sys
import numpy as np
import pandas as pd
import re
import json
import csv
import string
import time
from datetime import date, datetime
#
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Alignment, PatternFill, Font, Border, Side, NamedStyle

In [ ]:
stime = time.time() 

#  【樣本名冊: 清單Excel產出】

In [ ]:
# 目錄
folder_src = ['farmer_income_survey', 'resources']
folder_input = folder_src + ['input']
folder_output = folder_src + ['output']
year_this = '109'
# 今年的目錄
folder_input_thisyear  = folder_input + [year_this]
folder_output_thisyear = folder_output + [year_this]
# 
folder_csv             = folder_input_thisyear + ['csv']
folder_0_man           = folder_output_thisyear + ['樣本名冊_主選000【依調查員】']
folder_123_man         = folder_output_thisyear + ['樣本名冊_備選123【依調查員】']
folder_123_county      = folder_output_thisyear + ['樣本名冊_備選123【依縣市】']
# path
path_survey_csv        = os.path.join(*folder_csv, '01_調查名冊.csv')
path_folder_0_man      = os.path.join(*folder_0_man)
path_folder_123_man    = os.path.join(*folder_123_man)
path_folder_123_county = os.path.join(*folder_123_county)

In [ ]:
init = {
    # 'engine': 'python',
    'encoding': 'utf-8',
    'sep': ',',
    'keep_default_na': False,  # 放棄自動轉NA
}

# 1. 讀檔

## 1-1: 讀取【01_調查名冊.csv】四個欄位

In [ ]:
dtype = {'農戶編號': str}
# usecols = ['農戶編號', '主要經營型態', '可耕作地面積', '有無外僱']
# 109年取消外僱欄位
usecols = ['農戶編號', '主要經營型態', '可耕作地面積']
cols_map = {'主要經營型態': '經營種類'}
df_survey = pd.read_csv(path_survey_csv, **init, dtype=dtype, usecols=usecols).rename(columns=cols_map)
df_survey['有無外僱'] = '無'
#
print(df_survey.shape)
df_survey[:3]

## 1-2: 讀取【df_survey_combine.csv】

In [ ]:
dtype = {'farmer_num': str, 'link_num': str, 'tel': str, 'layer': str}
cols_map = {
    'farmer_num': '農戶編號',
    'link_num': '連結編號',
    'name': '戶長姓名',
    'tel': '電話',
    'addr': '地址',
    'layer': '層別',
    'man': 'man',
    'county': 'county',
}
#
df_survey_combine = pd.read_csv('df_survey_combine.csv', **init, dtype=dtype, usecols=cols_map.keys())
df_survey_combine = df_survey_combine.rename(columns=cols_map)[cols_map.values()]
df_survey_combine['樣本套號'] = df_survey_combine['連結編號'].str[-1]
#
print(df_survey_combine.shape)
df_survey_combine[:2]

## 1-3: join組成Excel所需的12欄(A-L)資料

In [ ]:
cols_n = 11
cols_12 = list(string.ascii_uppercase)[:cols_n]  # A-K
cols_14 = ['man', 'county'] + cols_12
#
cols_tar = [
    'man', 'county',
    '序號', '樣本套號', '農戶編號', '連結編號', '戶長姓名', '電話', '地址', '層別',
    '經營種類', '可耕作地面積', '成功打勾', 
    # '有無外僱'
]
#
df_survey_list = df_survey_combine.merge(df_survey, on='農戶編號', how='left').reindex(cols_tar, axis=1).fillna('')
df_survey_list = df_survey_list.sort_values(by=['man', '樣本套號', '地址']).reset_index(drop=True)
#
print(df_survey_list.shape)
df_survey_list[:3]

# 2. 樣本名冊處理

## 2-1: 輸出excel的函數

In [ ]:
def export_excel(df, path_folder, fn):
    # 儲存目錄
    if path_folder and not os.path.isdir(path_folder):
        mkdir(path_folder)
    # 儲存檔名
    path_csv = os.path.join(path_folder, f'{fn}.csv')
    path_xlsx = os.path.join(path_folder, f'{fn}.xlsx')
    # 先存csv
    df.to_csv(path_csv, index=False, header=False)
    # ==================================================================================
    # widths = [5.29, 5.29, 13.29, 9.29, 11.29, 12.29, 50.29, 4.29, 10.29, 20.29, 5.29, 5.29]
    widths = [5.29, 5.29, 13.29, 9.29, 11.29, 12.29, 50.29+5.29, 4.29, 10.29, 20.29, 5.29] # 109 改11欄
    ratio = 1.13
    # 格式
    title_fill = PatternFill(start_color='F7F7F7', end_color='F7F7F7', fill_type='solid')
    title_font = Font(bold=True)
    fillY = PatternFill(fill_type='solid', fgColor='FFFF00')
    align_center = Alignment(horizontal='center', vertical='center', wrap_text=True)
    align_right = Alignment(horizontal='right', vertical='center', wrap_text=True)
    align_left = Alignment(horizontal='left', vertical='center', wrap_text=True)
    align_top = Alignment(horizontal='left', vertical='top', wrap_text=True)
    side_medium = Side(style='medium')
    side_thin = Side(style='thin')
    border_4m = Border(top=side_medium, right=side_medium, bottom=side_medium, left=side_medium)
    border_4t = Border(top=side_thin, right=side_thin, bottom=side_thin, left=side_thin)
    # 樣式
    title = NamedStyle(name="title")
    title.alignment = align_center
    title.border = border_4t
    title.fill = title_fill
    title.font = title_font
    order_bold = NamedStyle(name="order_bold")
    order_bold.font = title_font
    order_bold.alignment = align_center
    order_bold.border = border_4t
    CB = NamedStyle(name="CB")
    CB.alignment = align_center
    CB.border = border_4t
    CBY = NamedStyle(name="CBY")
    CBY.alignment = align_center
    CBY.border = border_4t
    CBY.fill = fillY
    LB = NamedStyle(name="LB")
    LB.alignment = align_left
    LB.border = border_4t
    TB = NamedStyle(name="TB")
    TB.alignment = align_top
    TB.border = border_4t
    #
    wb = Workbook()
    sheet = wb.active
    for idx, col in enumerate(cols_12):
        sheet.column_dimensions[col].width = widths[idx]*ratio  # 調整欄寬
    # 從csv逐列貼資料到excel
    with open(path_csv, 'r', encoding='UTF-8') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        fillY_YN = -1  # 123套間隔加黃底，不一定有3，可能只有12
        link_num_pre = None
        for n, row in enumerate(csv_reader, 1):
            ##########################################
            sheet.append(row)
            ##########################################
            if n in [1, 2, 3]:
                sheet.merge_cells(start_row=n, end_row=n, start_column=1, end_column=cols_n)  # 從 A 欄併到 K 欄
                sheet.cell(row=n, column=1).alignment = (n < 3) and align_center or align_right  # 前兩列置中，第三列靠右
            elif n == 4:
                sheet.merge_cells(start_row=n, end_row=n, start_column=1, end_column=cols_n)
                for col in range(1, cols_n+1):
                    sheet.cell(row=n, column=col).border = border_4t  # 空白列
            elif n == 5:
                for col in range(1, cols_n+1):
                    sheet.cell(row=n, column=col).style = title  # 表頭列
            elif n > 5:
                sheet.row_dimensions[n].height = 1.95 * 17.153
                # 以連結編號前四碼判斷組別，同組123間隔加黃底，同縣市不一定有123，可能只有12，或只有3
                if row[3][-1] != '0':
                    if row[3][:4] != link_num_pre:
                        fillY_YN = -1*fillY_YN
                        link_num_pre = row[3][:4]
                sheet.cell(row=n, column=2).style = (fillY_YN > 0) and CBY or CB
                sheet.cell(row=n, column=1).style = order_bold
                for col in [11]:
                    sheet.cell(row=n, column=col).style = CB # 最後一欄要框線
                for col in [3, 4, 5, 6, 7, 8, 9, 10]:
                    sheet.cell(row=n, column=col).style = TB  # 戶長姓名/電話/地址靠上，多些空間註記
                # for col in [9, 10]:
                #     sheet.cell(row=n, column=col).style = LB

    #
    wb.save(path_xlsx)
    os.remove(path_csv)

In [ ]:
# 主備選excel前三列文字
title_a = f'{year_this}年主力農家所得調查樣本名冊─主選'
title_b = '本頁已完成調查戶數：_____'
title_c = '失敗戶請填寫失敗訪視紀錄表'
#
title_0   = [title_a, title_b, title_c]
title_123 = [title_a.replace('主選', '備選'), title_b, title_c]

## 2-2: 主選 0【依調查員】

In [ ]:
# 切出主選
where = df_survey_list['樣本套號'] == '0'
df_survey_list_0 = df_survey_list[where].sort_values(by=['man', '地址']).reset_index(drop=True)
#
# (1) 主要資料 type = 9 ___________________________________
# 地址排序後，每個調查員負責的農戶做rank當序號，欄位更新為A-L
df_survey_list_0[['序號']] = df_survey_list_0[['man']].reset_index().groupby('man').transform(lambda x: x.rank()).astype(int)  # 去小數點
df_survey_list_0 = df_survey_list_0.set_axis(cols_14, axis=1).assign(type=9)
# (2) 每個調查員excel的檔頭前5列 ___________________________________
df_man_0 = df_survey_list_0[['man']].drop_duplicates().reset_index(drop=True)
# A.前三列 type = 0
df_title_0 = df_man_0.assign(A=[title_0]*df_man_0.shape[0]).explode('A').reindex(cols_14, axis=1).fillna('').assign(type=0)
# B.第四列空白行 type = 1
df_space_0 = df_man_0.reindex(cols_14, axis=1).fillna('').assign(type=1)
# C.第五列粗黑標題 type = 2
df_cols_0 = df_man_0.reindex(cols_14, axis=1)
df_cols_0.iloc[:1, 2:] = cols_tar[2:]
df_cols_0 = df_cols_0.fillna(method='ffill').fillna('').assign(type=2)
# (3) 前五列+資料 ___________________________________
df_list = [df_survey_list_0, df_title_0, df_space_0, df_cols_0]
df_survey_list_0 = pd.concat(df_list).sort_values(by=['man', 'type'])
#
df_survey_list_0[:3]

In [ ]:
# 依調查員逐一切割主選df，並匯出excel
for man in df_man_0['man']:
    where = df_survey_list_0['man'] == man
    export_excel(df_survey_list_0[where][cols_12], path_folder_0_man, f'{man}_主選樣本名冊_{year_this}')

## 2-3: 備選 123【依調查員】

In [ ]:
# 切出備選123
where = df_survey_list['樣本套號'].str.contains('[123]', regex=True)
df_survey_list_123_src = df_survey_list[where]
df_survey_list_123 = df_survey_list_123_src.sort_values(by=['連結編號']).reset_index(drop=True)
# (1) 主要資料 type = 9 ___________________________________
# 123 新增以第一套為主的地址欄，2,3結尾者為na
# 連結編號排序，前四碼相同者為一組，第五碼2,3者，前向填充第五碼1的地址
df_survey_list_123['addr_1'] = np.where(df_survey_list_123['樣本套號'] == '1', df_survey_list_123['地址'], np.nan)
df_survey_list_123 = df_survey_list_123.fillna(method='ffill').sort_values(by=['man', 'addr_1', '連結編號']).reset_index(drop=True)
# addr_1/連結編號 排序後，每個調查員負責的農戶做rank當序號，欄位更新為A-L
df_survey_list_123[['序號']] = df_survey_list_123[['man']].reset_index().groupby('man').transform(lambda x: x.rank()).astype(int)  # 去小數點
del df_survey_list_123['addr_1']
df_survey_list_123 = df_survey_list_123.set_axis(cols_14, axis=1).assign(type=9)
# (2) 每個調查員excel的檔頭前5列 ___________________________________
df_man_123 = df_survey_list_123[['man']].drop_duplicates().reset_index(drop=True)
# A.前三列 type = 0
df_title_123 = df_man_123.assign(A=[title_123]*df_man_123.shape[0]).explode('A').reindex(cols_14, axis=1).fillna('').assign(type=0)
# B.第四列空白行 type = 1
df_space_123 = df_man_123.reindex(cols_14, axis=1).fillna('').assign(type=1)
# C.第五列粗黑標題 type = 2
df_cols_123 = df_man_123.reindex(cols_14, axis=1)
df_cols_123.iloc[:1, 2:] = cols_tar[2:]
df_cols_123 = df_cols_123.fillna(method='ffill').fillna('').assign(type=2)
# (3) 前五列+資料 ___________________________________
df_list = [df_survey_list_123, df_title_123, df_space_123, df_cols_123]
df_survey_list_123 = pd.concat(df_list).sort_values(by=['man', 'type'])
#
print(df_survey_list_123.shape)
df_survey_list_123[:3]

In [ ]:
# 依調查員逐一切割備選df，並匯出excel
for man in df_man_123['man']:
    where = df_survey_list_123['man'] == man
    export_excel(df_survey_list_123[where][cols_12], path_folder_123_man, f'{man}_備選123樣本名冊_{year_this}')

## 2-4: 備選 123【依縣市】

In [ ]:
df_survey_list_123 = df_survey_list_123_src.sort_values(by=['county', '連結編號']).reset_index(drop=True)
#
# (1) 主要資料 type = 9 ___________________________________
# 排序後，每個縣市的農戶做rank當序號，欄位更新為A-L
df_survey_list_123[['序號']] = df_survey_list_123[['county']].reset_index().groupby('county').transform(lambda x: x.rank()).astype(int)
df_survey_list_123 = df_survey_list_123.set_axis(cols_14, axis=1).assign(type=9)
# (2) 每個縣市 excel的檔頭前5列 ___________________________________
df_county_123 = df_survey_list_123[['county']].drop_duplicates().reset_index(drop=True)
# A.前三列 type = 0
df_title_123 = df_county_123.assign(A=[title_123]*df_county_123.shape[0]).explode('A').reindex(cols_14, axis=1).fillna('').assign(type=0)
# B.第四列空白行 type = 1
df_space_123 = df_county_123.reindex(cols_14, axis=1).fillna('').assign(type=1)
# C.第五列粗黑標題 type = 2
df_cols_123 = df_county_123.reindex(cols_14, axis=1)
df_cols_123.iloc[:1, 2:] = cols_tar[2:]
df_cols_123 = df_cols_123.fillna(method='ffill').fillna('').assign(type=2)
# (3) 前五列+資料 ___________________________________
df_list = [df_survey_list_123, df_title_123, df_space_123, df_cols_123]
df_survey_list_123 = pd.concat(df_list).sort_values(by=['county', 'type'])
#
print(df_survey_list_123.shape)
df_survey_list_123[:3]

In [ ]:
# 依縣市逐一切割備選df，並匯出excel
for county in df_county_123['county']:
    where = df_survey_list_123['county'] == county
    export_excel(df_survey_list_123[where][cols_12], path_folder_123_county, f'{county}_備選123樣本名冊_{year_this}')

In [ ]:
time.time() - stime